In [ ]:
"""
Sintetizador Microtonal Decimal (para Google Colaboratory)
--------------------------------------------------------------------------------
Este script genera un sintetizador musical interactivo en el navegador,
compatible con Google Colab. Utiliza Python para calcular las 240 frecuencias
del sistema microtonal decimal (20 tonos por octava, 12 octavas totales)
y luego inyecta HTML, CSS y JavaScript (usando la Web Audio API) para el
diseño, el sonido polifónico y la interacción.


El sistema de 240 tonos equidistantes y proporcionales se calcula con un ratio
de frecuencia r = 2^(1/20) para saltos de semitono/tono.

REQUISITOS:
1. Software: Un navegador web moderno (Chrome, Firefox).
2. Ambiente: Google Colaboratory (Colab) o cualquier entorno que soporte
   la ejecución de código Python y la renderización de HTML/JavaScript
   mediante IPython.display.
--------------------------------------------------------------------------------
"""
import math
import json
from IPython.display import display, HTML, Javascript

# ==============================================================================
# 1. CÁLCULO DE FRECUENCIAS DEL SISTEMA DECIMAL MICROTONAL (Python)
# ==============================================================================

# Definiciones del sistema:
TOTAL_SCALES = 12  # El usuario define 12 "escalas" análogas a octavas.
TONES_PER_SCALE = 20 # 10 teclas blancas + 10 teclas negras por escala.
TOTAL_TONES = TOTAL_SCALES * TONES_PER_SCALE  # 240 tonos equidistantes en total.
A4_440_HZ = 440.0 # Pivote y punto de referencia: La 440 Hz.
RATIO = math.pow(2, 1 / TONES_PER_SCALE) # Ratio de frecuencia entre tonos adyacentes.

# Asignamos A4_440_HZ al centro de la escala.
# El tono central (index 120) es A440, ya que hay 240 tonos totales.
# Tono de referencia A4 (440 Hz) será el tono número 120.
A4_INDEX = 120

# Calculamos todas las 240 frecuencias.
# El primer tono (índice 0) debe estar a 120 pasos por debajo del A4.
# Frecuencia del primer tono (Índice 0)
f_0 = A4_440_HZ / math.pow(RATIO, A4_INDEX)

# Generamos el array de frecuencias
frequencies = []
for i in range(TOTAL_TONES):
    f = f_0 * math.pow(RATIO, i)
    frequencies.append(round(f, 3))

# La escala de visualización activa se compone de 40 tonos (2 escalas completas)
# Las escalas se numeran del 1 al 12.

# Mapeo de teclas de la computadora a índices de tono (para 2 escalas = 40 tonos)
# El mapeo se define para las escalas *activas* (Scale N y Scale N+1)
KEY_MAP_1 = {
    # ESCALA N (Teclas Negras: 1.5, 2.5... 10.5)
    '1': 1, '2': 3, '3': 5, '4': 7, '5': 9, '6': 11, '7': 13, '8': 15, '9': 17, '0': 19,
    # ESCALA N (Teclas Blancas: 1, 2... 10)
    'q': 0, 'w': 2, 'e': 4, 'r': 6, 't': 8, 'y': 10, 'u': 12, 'i': 14, 'o': 16, 'p': 18,
}
KEY_MAP_2 = {
    # ESCALA N+1 (Teclas Negras: 1.5, 2.5... 10.5)
    'a': 21, 's': 23, 'd': 25, 'f': 27, 'g': 29, 'h': 31, 'j': 33, 'k': 35, 'l': 37, 'ñ': 39,
    # ESCALA N+1 (Teclas Blancas: 1, 2... 10)
    'z': 20, 'x': 22, 'c': 24, 'v': 26, 'b': 28, 'n': 30, 'm': 32, ',': 34, '.': 36, '-': 38,
}
KEY_MAP = {**KEY_MAP_1, **KEY_MAP_2} # Combinamos los mapas

# Preparamos las variables Python para inyección
frequencies_json = json.dumps(frequencies)
key_map_json = json.dumps(KEY_MAP)

# ==============================================================================
# 2. DEFINICIÓN DE HTML/CSS/JAVASCRIPT (Web Audio API)
# Se usa .format() para evitar el error de las llaves anidadas en PyCharm.
# Las plantillas JS (${...}) se han reemplazado por concatenación de strings ('...'+var)
# para evitar conflictos con el linter de Python.
# ==============================================================================

HTML_TEMPLATE = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;700&display=swap');
    body {{ font-family: 'Inter', sans-serif; background-color: #f7f7f7; display: flex; flex-direction: column; align-items: center; padding: 20px; }}
    .container {{ width: 100%; max-width: 900px; background: #fff; border-radius: 12px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1); padding: 20px; text-align: center; }}
    .controls-panel {{ display: flex; justify-content: space-around; align-items: center; margin-bottom: 20px; background: #eee; padding: 10px; border-radius: 8px; }}
    .scale-display {{ font-size: 1.5rem; font-weight: bold; padding: 5px 15px; border-radius: 6px; background-color: #333; color: white; min-width: 100px; text-align: center; }}
    .control-button {{ background-color: #007bff; color: white; border: none; padding: 10px 15px; border-radius: 8px; cursor: pointer; transition: all 0.2s; font-size: 1.2rem; margin: 0 10px; }}
    .control-button:hover {{ background-color: #0056b3; transform: scale(1.05); }}
    .control-button:disabled {{ background-color: #aaa; cursor: not-allowed; }}
    .keyboard {{ display: flex; position: relative; height: 180px; margin-top: 30px; border: 2px solid #333; border-radius: 8px; overflow: hidden; }}
    .key-white {{ background-color: #fff; border: 1px solid #ccc; width: 45px; height: 100%; position: relative; margin-left: -1px; display: flex; justify-content: center; align-items: flex-end; padding-bottom: 5px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); transition: background-color 0.1s; }}
    .key-black {{ background-color: #333; width: 30px; height: 60%; position: absolute; z-index: 2; margin-left: -15px; display: flex; justify-content: center; align-items: flex-start; padding-top: 5px; color: #fff; font-size: 0.7rem; border-radius: 0 0 4px 4px; box-shadow: 0 2px 6px rgba(0, 0, 0, 0.5); transition: background-color 0.1s; }}
    .key-info {{ position: absolute; top: 10px; font-size: 0.6rem; color: #666; }}
    .pressed {{ background-color: #ffcc00 !important; box-shadow: 0 0 10px #ffcc00; }}
    .black-pressed {{ background-color: #666 !important; box-shadow: 0 0 10px #ffcc00; }}
    .tone-label {{ font-size: 0.6rem; color: #fff; }}

    /* Estilos del Overlay de Inicio */
    #startOverlay {{
        position: absolute; top: 0; left: 0; right: 0; bottom: 0;
        background: rgba(255, 255, 255, 0.95); z-index: 1000;
        display: flex; justify-content: center; align-items: center;
        flex-direction: column; border-radius: 12px;
        text-align: center;
    }}
    #startButton {{
        padding: 15px 30px; font-size: 1.2rem; background-color: #10b981;
        color: white; border: none; border-radius: 8px; cursor: pointer;
        transition: background-color 0.3s; box-shadow: 0 4px #047857;
    }}
    #startButton:active {{ transform: translateY(2px); box-shadow: 0 2px #047857; }}
</style>

<div class="container" id="mainContainer">
    <h2>Sintetizador Decimal Microtonal (240 Tonos)</h2>

    <div class="controls-panel">
        <label for="waveSelector">Tipo de Onda:</label>
        <select id="waveSelector" class="p-2 border rounded-md">
            <option value="sine">Sinusoidal</option>
            <option value="square">Cuadrada</option>
            <option value="sawtooth">Diente de Sierra</option>
            <option value="triangle">Triangular</option>
        </select>

        <button id="prevScale" class="control-button" disabled>&larr; Escala Anterior</button>
        <div class="scale-display">Escalas <span id="currentScales">1 y 2</span></div>
        <button id="nextScale" class="control-button" disabled>Escala Siguiente &rarr;</button>
    </div>

    <p>Teclas Blancas: Fila QWERTY y ZXC. Teclas Negras: Fila Numérica y ASDF.</p>

    <div id="keyboard" class="keyboard">
        <!-- Teclas se insertarán aquí por JavaScript -->
    </div>

    <div id="statusMessage" style="margin-top: 15px; color: green;"></div>

    <div id="startOverlay">
        <h3>Activar Sintetizador</h3>
        <p>El navegador requiere un click para iniciar el sistema de audio.</p>
        <button id="startButton">INICIAR (Click para Activar Audio)</button>
        <p style="font-size: 0.8rem; margin-top: 10px;">(Esto resuelve el bloqueo de audio por seguridad del navegador/filtros)</p>
    </div>
</div>

<script>
    // JSON de frecuencias generado por Python
    const ALL_FRECUENCIAS = {0};
    const KEY_MAP = {1};
    const TOTAL_SCALES = {2};
    const TONES_PER_SCALE = {3};

    // Inicializamos a null. Se creará en el primer click.
    let audioContext = null;
    let oscillators = {{}}; // Objeto para mantener los osciladores activos (para polifonía)
    let currentStartScale = 1; // La primera escala visible (de 1 a 11)
    let currentWaveType = 'sine';

    // Asigna los elementos del DOM
    const keyboardContainer = document.getElementById('keyboard');
    const displayElement = document.getElementById('currentScales');
    const prevButton = document.getElementById('prevScale');
    const nextButton = document.getElementById('nextScale');
    const waveSelector = document.getElementById('waveSelector');
    const startButton = document.getElementById('startButton');
    const startOverlay = document.getElementById('startOverlay');
    const statusMessage = document.getElementById('statusMessage');

    // ==========================================================================
    // LÓGICA DE INICIALIZACIÓN SEGURA (Solución al Bloqueo)
    // ==========================================================================

    function initAudioContext() {{
        if (audioContext) return;

        audioContext = new (window.AudioContext || window.webkitAudioContext)();

        // 🚨 CORRECCIÓN CLAVE: Forzar la reanudación del contexto de audio
        // para asegurar que pase de 'suspended' a 'running'.
        if (audioContext.state === 'suspended') {{
            audioContext.resume().then(() => {{
                console.log('AudioContext: reanudado exitosamente.');
            }});
        }}

        // Remover el overlay y habilitar controles
        startOverlay.style.display = 'none';

        // Habilitar botones de navegación (solo si no están en el límite)
        navigateScales(0); // Esto fuerza la actualización del estado de los botones

        statusMessage.textContent = '¡Audio Activo! Use el ratón o el teclado (QWERTY, ASDF, ZXC, Números) para tocar.';

        // Agregar listeners de teclado DESPUÉS de activar el audio
        document.addEventListener('keydown', handleKeyDown);
        document.addEventListener('keyup', handleKeyUp);
    }}

    startButton.addEventListener('click', initAudioContext);


    // Inicializa el tipo de onda
    waveSelector.addEventListener('change', (e) => {{
        currentWaveType = e.target.value;
    }});

    // Función principal para generar y dibujar el teclado
    function drawKeyboard() {{
        keyboardContainer.innerHTML = '';

        const scaleN = currentStartScale;
        const scaleN_plus_1 = currentStartScale + 1;
        const startIndex = (scaleN - 1) * TONES_PER_SCALE;

        // CORRECCIÓN: Usamos concatenación simple para evitar errores de linter en PyCharm
        displayElement.textContent = scaleN + ' y ' + scaleN_plus_1;
        prevButton.disabled = (scaleN === 1);
        nextButton.disabled = (scaleN === TOTAL_SCALES - 1);

        // Genera las 40 teclas
        let whiteKeyPosition = 0;

        for (let i = 0; i < TONES_PER_SCALE * 2; i++) {{
            const frequencyIndex = startIndex + i;
            const freq = ALL_FRECUENCIAS[frequencyIndex];
            const isBlackKey = i % 2 !== 0;

            const toneNumber = Math.ceil((i + 1) / 2);
            const currentScale = (i < TONES_PER_SCALE) ? scaleN : scaleN_plus_1;

            let keyElement = document.createElement('div');
            keyElement.dataset.freq = freq;
            keyElement.dataset.index = frequencyIndex;

            if (!isBlackKey) {{
                // TECLA BLANCA
                keyElement.className = 'key-white';
                keyElement.style.marginLeft = (whiteKeyPosition === 0) ? '0' : '-1px';

                // CORRECCIÓN: Usamos concatenación simple
                keyElement.innerHTML =
                    '<span class="key-info">' + currentScale + ' (' + toneNumber + ')</span>' +
                    '<span style="font-size: 0.8rem; font-weight: bold;">' + freq + ' Hz</span>';

                keyboardContainer.appendChild(keyElement);
                whiteKeyPosition++;
            }} else {{
                // TECLA NEGRA
                keyElement.className = 'key-black';

                // Posicionamiento
                const totalWhiteWidth = whiteKeyPosition * 45;
                const offset = -15;

                // CORRECCIÓN: Usamos concatenación simple
                keyElement.style.left = (totalWhiteWidth + offset) + 'px';
                keyElement.innerHTML =
                    '<span class="tone-label">' + currentScale + ' (' + toneNumber + '.5)</span>' +
                    '<span class="tone-label" style="margin-top: 15px;">' + freq + '</span>';

                keyboardContainer.appendChild(keyElement);
            }}

            // Agregar listeners de mouse/touch a todas las teclas
            keyElement.addEventListener('mousedown', () => startTone(freq, frequencyIndex, keyElement));
            keyElement.addEventListener('mouseup', () => stopTone(frequencyIndex, keyElement));
            keyElement.addEventListener('mouseout', () => stopTone(frequencyIndex, keyElement));
            keyElement.addEventListener('touchstart', (e) => {{ e.preventDefault(); startTone(freq, frequencyIndex, keyElement); }});
            keyElement.addEventListener('touchend', (e) => {{ e.preventDefault(); stopTone(frequencyIndex, keyElement); }});
        }}
    }}

    // ==========================================================================
    // LÓGICA DE AUDIO Y POLIFONÍA (Web Audio API)
    // ==========================================================================

    function startTone(frequency, index, element) {{
        if (!audioContext) {{
            statusMessage.textContent = 'ERROR: Presione el botón INICIAR para activar el AudioContext.';
            return;
        }}
        if (oscillators[index]) return;

        // 1. Crear Oscilador
        let oscillator = audioContext.createOscillator();
        oscillator.type = currentWaveType;
        oscillator.frequency.setValueAtTime(frequency, audioContext.currentTime);

        // 2. Crear Ganancia (Control de Volumen)
        let gainNode = audioContext.createGain();
        gainNode.gain.setValueAtTime(0, audioContext.currentTime);
        gainNode.gain.linearRampToValueAtTime(0.5, audioContext.currentTime + 0.02);

        // 3. Conexión y Reproducción
        oscillator.connect(gainNode);
        gainNode.connect(audioContext.destination);

        oscillators[index] = {{ osc: oscillator, gain: gainNode }};

        oscillator.start();

        // 4. Feedback Visual
        const pressedClass = element.classList.contains('key-black') ? 'black-pressed' : 'pressed';
        element.classList.add(pressedClass);
    }}

    function stopTone(index, element) {{
        const keyInfo = oscillators[index];
        if (!keyInfo) return;

        // Liberar
        keyInfo.gain.gain.cancelScheduledValues(audioContext.currentTime);
        keyInfo.gain.gain.linearRampToValueAtTime(0, audioContext.currentTime + 0.1);

        keyInfo.osc.stop(audioContext.currentTime + 0.1);

        delete oscillators[index];
        element.classList.remove('pressed', 'black-pressed');
    }}

    // ==========================================================================
    // LÓGICA DE NAVEGACIÓN (Flechas y Teclado)
    // ==========================================================================

    function navigateScales(direction) {{
        const newStartScale = currentStartScale + direction;

        if (newStartScale >= 1 && newStartScale <= TOTAL_SCALES - 1) {{
            currentStartScale = newStartScale;
        }} else if (direction === 0) {{
             // No hacemos nada, solo forzamos el redibujo para habilitar/deshabilitar botones
        }}

        // Redibujar y actualizar el estado de los botones
        drawKeyboard();
        prevButton.disabled = (currentStartScale === 1);
        nextButton.disabled = (currentStartScale === TOTAL_SCALES - 1);
    }}

    prevButton.addEventListener('click', () => navigateScales(-1));
    nextButton.addEventListener('click', () => navigateScales(1));


    // Mapeo de teclas de computadora a tonos
    let activeKeys = new Set();

    const handleKeyDown = (e) => {{
        if (!audioContext) return; // Ignorar si el audio no está activo
        if (activeKeys.has(e.key)) return;
        activeKeys.add(e.key);

        // Control de navegación (Flechas)
        if (e.key === 'ArrowLeft') {{
            navigateScales(-1);
            return;
        }}
        if (e.key === 'ArrowRight') {{
            navigateScales(1);
            return;
        }}

        // Control de Tonos
        const toneOffset = KEY_MAP[e.key.toLowerCase()];

        if (toneOffset !== undefined) {{
            const globalIndex = ((currentStartScale - 1) * TONES_PER_SCALE) + toneOffset;
            const freq = ALL_FRECUENCIAS[globalIndex];

            const keyElement = document.querySelector('[data-index="' + globalIndex + '"]');
            if (keyElement) {{
                startTone(freq, globalIndex, keyElement);
            }}
        }}
    }};

    const handleKeyUp = (e) => {{
        activeKeys.delete(e.key);
        const toneOffset = KEY_MAP[e.key.toLowerCase()];

        if (toneOffset !== undefined) {{
            const globalIndex = ((currentStartScale - 1) * TONES_PER_SCALE) + toneOffset;
            const keyElement = document.querySelector('[data-index="' + globalIndex + '"]');
            if (keyElement) {{
                stopTone(globalIndex, keyElement);
            }}
        }}
    }};

    // Inicializar la interfaz visual (el overlay aparecerá primero)
    drawKeyboard();
</script>
""".format(frequencies_json, key_map_json, TOTAL_SCALES, TONES_PER_SCALE) # <--- Uso de .format()

# ==============================================================================
# 3. EJECUCIÓN DEL SCRIPT (Python)
# ==============================================================================

print("--- INSTRUCTIVO DE EJECUCIÓN PARA COLAB ---")
print("1. Copie todo el código de este bloque y péguelo en una celda de código de Google Colaboratory.")
print("2. Presione CTRL+ENTER o el botón de 'Play' para ejecutar la celda.")
print("3. La interfaz del teclado aparecerá debajo de la celda de código.")
print("4. **Paso Crucial:** Presione el botón 'INICIAR' en el teclado para activar el sonido.")
print("5. Requisitos de Software: Solo necesita un navegador moderno con soporte para Web Audio API (Chrome, Firefox, Edge).")
print("6. Para tocar: Use el ratón o el teclado de su computadora, incluyendo las flechas Izquierda/Derecha para navegar las escalas.")
print("-------------------------------------------\n")

# Mostrar el HTML/JS en la salida de Colab
display(HTML(HTML_TEMPLATE))

--- INSTRUCTIVO DE EJECUCIÓN PARA COLAB ---
1. Copie todo el código de este bloque y péguelo en una celda de código de Google Colaboratory.
2. Presione CTRL+ENTER o el botón de 'Play' para ejecutar la celda.
3. La interfaz del teclado aparecerá debajo de la celda de código.
4. **Paso Crucial:** Presione el botón 'INICIAR' en el teclado para activar el sonido.
5. Requisitos de Software: Solo necesita un navegador moderno con soporte para Web Audio API (Chrome, Firefox, Edge).
6. Para tocar: Use el ratón o el teclado de su computadora, incluyendo las flechas Izquierda/Derecha para navegar las escalas.
-------------------------------------------

